In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

On récupère les données qui ont été tansformées

In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_train.csv', index_col='TransactionID')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_test.csv', index_col='TransactionID')
Y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/Y_train.csv', index_col='TransactionID')

print("loading successful!")

loading successful!


Colonne que l'on va mettre en index

In [4]:
df = test_data.reset_index(inplace=False)
df.head(5)
colTransactionID = df["TransactionID"]

# KNN

In [5]:
from sklearn.neighbors import KNeighborsClassifier

#instanciation
knn = KNeighborsClassifier(n_neighbors=10)
print("model done")

#training
knn.fit(train_data.iloc[0:10000], Y_train.iloc[0:10000].values.ravel()) #on ne prend pas toutes les lignes sinon ca plante
print("training done")

#prediction 
predictionKnn = pd.DataFrame(knn.predict(test_data))
print("prediction done")


model done
training done
prediction done


In [6]:
predictionfinalKnn = pd.concat([colTransactionID, predictionKnn], axis=1)
predictionfinalKnn.head(5)
predictionfinalKnn.columns = ["TransactionID", "isFraud"]
predictionfinalKnn.set_index(["TransactionID"], inplace = True)
predictionfinalKnn.to_csv("/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/submitKNN")

In [7]:
predictionfinalKnn["isFraud"].value_counts()

0    505643
1      1048
Name: isFraud, dtype: int64

result submission (neightboors = 5) : 0.533

# SVM

### SVM LINEAR

In [8]:
from sklearn.svm import SVC
#instanciation
model_SVC = SVC(kernel= 'linear', gamma='scale', shrinking=True)
print("model done")

#training
model_SVC.fit(train_data.iloc[0:10000], Y_train.iloc[0:10000].values.ravel())
print("training done")

#prediction 
predictionSVM = pd.DataFrame(model_SVC.predict(test_data))
print("prediction done")

model done
training done
prediction done


In [9]:
predictionfinalSVM = pd.concat([colTransactionID, predictionSVM], axis=1)
predictionfinalSVM.head(5)
predictionfinalSVM.columns = ["TransactionID", "isFraud"]
predictionfinalSVM.set_index(["TransactionID"], inplace = True)
predictionfinalSVM.to_csv("/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/submitSVM")

In [10]:
predictionfinalSVM["isFraud"].value_counts()

0    493584
1     13107
Name: isFraud, dtype: int64

submission result : 0.618

### SVM POLY

In [11]:
from sklearn.svm import SVC

#instanciation
model_SVC = SVC(kernel= 'poly', degree= 7, gamma='scale', shrinking=True)
print("model done")

#training
model_SVC.fit(train_data.iloc[0:10000], Y_train.iloc[0:10000].values.ravel())
print("training done")

#prediction 
predictionSVMPoly = pd.DataFrame(model_SVC.predict(test_data))
print("prediction done")

model done
training done
prediction done


In [12]:
predictionfinalSVMPoly = pd.concat([colTransactionID, predictionSVMPoly], axis=1)
predictionfinalSVMPoly.head(5)
predictionfinalSVMPoly.columns = ["TransactionID", "isFraud"]
predictionfinalSVMPoly.set_index(["TransactionID"], inplace = True)
predictionfinalSVMPoly.to_csv("/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/submitSVMPoly")

In [13]:
predictionfinalSVMPoly["isFraud"].value_counts()

0    500442
1      6249
Name: isFraud, dtype: int64

result submission (degree = 3) : 0.592

result submission (degree = 7) : 0.586

# LOGISTIC REGRESSION

In [14]:
from sklearn.linear_model import LogisticRegression

# SET UP
lr = LogisticRegression(penalty='l2', max_iter=500, n_jobs=6, tol=1e-6, solver="sag")
print("set up done")

#training
lr.fit(train_data.iloc[0:10000], Y_train.iloc[0:10000].values.ravel())
print("training done")

#prediction
Yhat_lr = lr.predict_proba(test_data)
print("prediction done")

set up done


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


training done
prediction done


In [15]:
predictionLinearRegression = pd.DataFrame(Yhat_lr[:,1])

In [16]:
predictionfinalLinearRegression = pd.concat([colTransactionID, predictionLinearRegression], axis=1)
predictionfinalLinearRegression.head(5)
predictionfinalLinearRegression.columns = ["TransactionID", "isFraud"]
predictionfinalLinearRegression.set_index(["TransactionID"], inplace = True)
predictionfinalLinearRegression.to_csv("/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/submitLinearRegression")

In [17]:
predictionfinalLinearRegression["isFraud"].value_counts()

1.000000    303
1.000000     16
1.000000     12
1.000000     10
1.000000      5
           ... 
0.212222      1
0.004041      1
0.009787      1
0.511037      1
0.271174      1
Name: isFraud, Length: 506339, dtype: int64

result submission : 0.781

# RANDOM FOREST

In [18]:
from sklearn.ensemble import RandomForestClassifier

#initialisation
treeCount = 1000
m = 15
rf = RandomForestClassifier(max_features=m, min_samples_leaf=1, n_estimators=treeCount)
print("end of initialisation")

#training
rf.fit(train_data.iloc[20000:30000], Y_train.iloc[20000:30000].values.ravel())
print("end of training")

#prediction
Yhat_rf = rf.predict_proba(test_data)
print("end of prediction")


end of initialisation
end of training
end of prediction


In [19]:
predictionRandomForest = pd.DataFrame(Yhat_rf[:,1])

In [20]:
predictionfinalRandomForest = pd.concat([colTransactionID, predictionRandomForest], axis=1)
predictionfinalRandomForest.head(5)
predictionfinalRandomForest.columns = ["TransactionID", "isFraud"]
predictionfinalRandomForest.set_index(["TransactionID"], inplace = True)
predictionfinalRandomForest.to_csv("/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/submitRandomForest")

In [21]:
predictionfinalRandomForest["isFraud"].value_counts()

0.117    4135
0.114    4044
0.112    4041
0.116    4039
0.115    4018
         ... 
0.903       1
0.905       1
0.910       1
0.904       1
0.915       1
Name: isFraud, Length: 913, dtype: int64

result from submission : 0.87